In [12]:
import requests
import OSMPythonTools as osm

In [52]:
from OSMPythonTools.api import Api
api = Api()
way = api.query('way/5887599')
way

In [22]:
from OSMPythonTools.api import Api
api = Api()
way = api.query('way/5887599')


print(way.tag('building'))
print(way.tag('architect'))
print(way.tag('website'))

castle
Johann Lucas von Hildebrandt
https://www.belvedere.at/


In [227]:
from OSMPythonTools.overpass import Overpass

overpass = Overpass()
result = overpass.query('nwr["addr:city"="Kraków"]["tourism"]; out;')

s = set({})
for dupa in result.elements():
  # node: dupa.lat(), dupa.lon()
  print(dupa)

In [67]:
from OSMPythonTools.overpass import Overpass, overpassQueryBuilder

overpass = Overpass()
result = overpassQueryBuilder()

0

In [311]:
from OSMPythonTools.nominatim import Nominatim
import json

def user_search(x:float, y:float) -> str:
    # 1° of latitude = always 111.32 km
    eps = 0.0005
    x0, x1 = x - eps, x + eps
    y0, y1 = y - eps, y + eps

    overpass = Overpass()
    result = overpass.query(f'nwr["tourism"]({x0},{y0},{x1},{y1}); out;')

    s = set({})
    for obj in result.elements():
        name = obj.tag("name")
        description = gen_description(obj)
        address = gen_address(obj)
        category = obj.tag("tourism")
        latitide, longitude = get_lat_lon(obj)
        
        picture_url = ""

        if None in (name, address, category, latitide, longitude, picture_url):
            continue
        s.add((name, description, address, category.capitalize(),
              latitide, longitude, picture_url))
    return s

user_search(50.0641425, 19.9231397, "sklep")

{('Muzeum Geologiczne WGGiOŚ AGH',
  '',
  '30-059 Kraków, Aleja Adama Mickiewicza 30',
  'Museum',
  50.0641425,
  19.9231397,
  '')}

## Fajne funkcje

In [339]:
def gen_description(obj):
  txt = ""
  if not (website := obj.tag("website")) is None:
    txt += "Witryna internetowa: " + website + "\n"
  if not (hours := obj.tag("opening_hours")) is None:
    txt += "Godziny otwarcia: " + hours + "\n"
  if not (wheelchair := obj.tag("wheelchair")) is None:
    txt += "Przystosowanie dla osób niepełnosprawnych: " + wheelchair + "\n"
  if not (fee := obj.tag("fee")) is None:
    txt += "Opłaty: " + fee + "\n"
  if not (phone := obj.tag("phone")) is None:
    txt += "Telefon: " + phone + "\n"
  if not (address := gen_address(obj)) is None:
    txt += "Adres: " + address  + "\n"
    
  return txt

def gen_address(obj):
  r = [ obj.tag("addr:postcode"), " ", obj.tag("addr:city"), ", ", obj.tag("addr:street"), " ", obj.tag("addr:housenumber") ]
  for i in r:
    if i is None:
      return None
  return "".join(r)
  
def get_lat_lon(obj):
  if not obj.lat() is None and not obj.lon() is None:
    return obj.lat(), obj.lon()
  elif len(obj.nodes()) == 0:
    return  None, None
  elif not obj.nodes()[0].lat() is None and not obj.nodes()[0].lon() is None:
    return obj.nodes()[0].lat(), obj.nodes()[0].lon()
  else:
    return None, None

def search_for_cool_objects(city):
    # cools found in Krakow: hotel hostel information motel gallery camp_site theme_park apartment zoo attraction guest_house museum
    COOLS = {"information", "gallery", "camp_site",
             "theme_park", "zoo", "attraction", "museum"}

    overpass = Overpass()
    result = overpass.query(f'nwr["addr:city"="{city}"]["tourism"]; out;')

    s = set({})
    for obj in result.elements():
        if not obj.tag("tourism") in COOLS:
            continue

        name = obj.tag("name")
        description = gen_description(obj)
        address = gen_address(obj)
        category = obj.tag("tourism")
        latitide, longitude = get_lat_lon(obj)
        
        picture_url = ""

        if None in (name, address, category, latitide, longitude, picture_url):
            continue
        s.add((name, description, address, category.capitalize(),
              latitide, longitude, picture_url))
    return s


search_for_cool_objects("Kraków")

{('Arka Pana',
  'Witryna internetowa: http://www.arkapana.pl/\nAdres: 31-831 Kraków, Obrońców Krzyża 1\n',
  '31-831 Kraków, Obrońców Krzyża 1',
  'Attraction',
  50.0851084,
  20.0295468,
  ''),
 ('Autorska Galeria Rzeźby prof. Bronisława Chromego',
  'Witryna internetowa: http://www.bronislawchromy.pl/index.html\nAdres: 30-223 Kraków, Krańcowa 4\n',
  '30-223 Kraków, Krańcowa 4',
  'Gallery',
  50.0652802,
  19.8708249,
  ''),
 ('Balon',
  'Witryna internetowa: http://balonwidokowy.pl/\nTelefon: +48 500 350 505\nAdres: 30-302 Kraków, Bulwar Wołyński 2\n',
  '30-302 Kraków, Bulwar Wołyński 2',
  'Attraction',
  50.0463192,
  19.9357564,
  ''),
 ('Bunkier Sztuki',
  'Witryna internetowa: http://bunkier.art.pl/\nGodziny otwarcia: Tu-Su 11:00-19:00\nAdres: 31-011 Kraków, Plac Szczepański 3a\n',
  '31-011 Kraków, Plac Szczepański 3a',
  'Museum',
  50.0635805,
  19.9344469,
  ''),
 ('Camping Smok nr46',
  'Witryna internetowa: http://smok.krakow.pl/pl\nOpłaty: yes\nTelefon: +48 12 429 83

In [345]:
a = [(6, 1),(5, 5),(4, 3),(3, 2),(2, 7),]
sorted(a, key=lambda x: x[0]*x[1])

[(6, 1), (3, 2), (4, 3), (2, 7), (5, 5)]

## Photos

In [346]:
import requests

In [355]:
r = requests.get("https://routing.openstreetmap.de/routed-car/route/v1/driving/19.910908646685904,50.0832341;19.913457949864238,50.08375765?overview=false&geometries=polyline&steps=true&hints=t_ObgPj93YUhAAAAYQAAABIAAAAAAAAAKzbyQfhIq0IrpYFBAAAAACEAAABhAAAAEgAAAAAAAAD86AAAadAvAbs1_AL90C8BojX8AgEA7wBWyR-F%3BtP3dhQX-3YVfAAAAZgAAAAAAAAAAAAAAvZyqQmKss0IAAAAAAAAAAF8AAABmAAAAAAAAAAAAAAD86AAAkuQvAdQ3_ALn4y8B6zf8AgAA_wtWyR-F")
r.content.decode("utf-8")

'{"code":"Ok","waypoints":[{"hint":"t_ObgPj93YUhAAAAYQAAABIAAAAAAAAAKzbyQfhIq0IrpYFBAAAAACEAAABhAAAAEgAAAAAAAAD86AAAadAvAbs1_AL90C8BojX8AgEA7wBWyR-F","distance":10.953654,"location":[19.910761,50.083259],"name":"Murarska"},{"hint":"jf3dhf___38MAAAAGgAAABIAAAAnAQAADoc2QQ6HNkGlbn5Bq_eCQwwAAAAaAAAAEgAAACcBAAD86AAAc9svAVg3_ALy2i8Brjf8AgEA7wJWyR-F","distance":13.296025,"location":[19.913587,50.083672],"name":"Wacława Nałkowskiego"}],"routes":[{"legs":[{"steps":[{"intersections":[{"out":0,"entry":[true],"location":[19.910761,50.083259],"bearings":[195]}],"driving_side":"right","geometry":"k{tpHgyoxBr@TZH","duration":10.7,"distance":46.5,"name":"Murarska","weight":10.7,"mode":"driving","maneuver":{"bearing_after":195,"location":[19.910761,50.083259],"type":"depart","bearing_before":0,"modifier":"left"}},{"intersections":[{"out":1,"in":0,"entry":[false,true,true],"location":[19.910595,50.082855],"bearings":[15,105,285]},{"out":1,"in":2,"entry":[true,true,false],"location":[19.911238,50.082763]

In [369]:
from io import BytesIO, TextIOWrapper
from google_images_download import google_images_download
import sys


old_stdout = sys.stdout
sys.stdout = TextIOWrapper(BytesIO(), sys.stdout.encoding)


response = google_images_download.googleimagesdownload()

arguments = {
    "keywords": "carrots",
    "limit": 3,
    "print_urls": True,
    "size": "large",
}
paths = response.download(arguments)

sys.stdout.seek(0)
output = sys.stdout.read()

sys.stdout.close()
sys.stdout = old_stdout

for line in output.split("\n"):
    if line.startswith("Image URL:"):
        print(line.replace("Image URL: ", ""))

In [372]:
def get_photos_from_google(key: str):
  old_stdout = sys.stdout
  sys.stdout = TextIOWrapper(BytesIO(), sys.stdout.encoding)

  response = google_images_download.googleimagesdownload()

  arguments = {
      "keywords": key,
      "limit": 5,
      "print_urls": True,
  }
  paths = response.download(arguments)

  sys.stdout.seek(0)
  output = sys.stdout.read()

  sys.stdout.close()
  sys.stdout = old_stdout

  result = []
  for line in output.split("\n"):
      if line.startswith("Image URL:"):
          result.append(line.replace("Image URL: ", ""))
  return result
        

In [373]:
get_photos_from_google("marchewka")

[]